In [1]:
%matplotlib inline
from IPython.display import Image
import numpy.matlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats
import openpyxl
from scipy.optimize import curve_fit
from mpl_toolkits.mplot3d import Axes3D

import xml.etree.ElementTree as ET
from subprocess import call
import vtk
from vtk.util.numpy_support import vtk_to_numpy
from scipy.interpolate import griddata

import openmc

import Define_Nektar
import Define_OpenMC

import os
import time

import warnings
warnings.filterwarnings("ignore")

In [4]:
# Parameters of reactor
# Unit: cm
# To be discussed: Parameters for reactor to become critical
parameters_dic = {}

parameters_dic.update(fuel_r = 11/2)
parameters_dic.update(fuel_h = 19.5)

parameters_dic.update(controlRod_r = 4.4/2)
parameters_dic.update(controlRod_h_max = 27)
parameters_dic.update(controlRod_l = 24.5)

parameters_dic.update(reflector_r = 42/2)

parameters_dic.update(heat_pipe_R = 1.27/2)
parameters_dic.update(heat_pipe_r = 1.27/2-0.1)

parameters_dic.update(top_distance = 10.5)
parameters_dic.update(bottom_distance = 5)

parameters_dic.update(heat_power = 4000)

parameters_dic.update(reflector_h = parameters_dic['top_distance']+parameters_dic['bottom_distance']+parameters_dic['fuel_h'])

temp_pipe = 1073.5
# Insert control rod, maximum is 27
controlRod_deep = 0
empty_reflector_height = 5
# Number of cells
cells_num_dic = {'n_r':10,'n_r_outer':5,'n_h':20}
# Settings of OpenMC
settings_dic = {'batches':20,'inactive':10,'particles':1000}
#Settings of Nektar++
file_name = 'HeatPipeReactor_3' # fiel_name.xml with settings of Poisson solver
solver_name = 'ADRSolver' 
# Number of iteration
iteration = 10
# Initial temperature distribution in cells
Initial_temperature = 1073.5
temp_cells_mat = Initial_temperature*np.ones((cells_num_dic['n_h'],(cells_num_dic['n_r_outer']+cells_num_dic['n_r'])))
# Get some data
volume_mat,fuel_cell_ID_list = Define_OpenMC.define_Geo_Mat_Set(cells_num_dic,parameters_dic,settings_dic,temp_cells_mat,controlRod_deep,empty_reflector_height)

In [5]:
openmc.run(output=False)
sp = openmc.StatePoint('statepoint.{}.h5'.format(settings_dic['batches']))
k_eff = sp.k_combined
del sp
# k_eff_mean_vec[i] = k_eff.nominal_value
# k_eff_dev_vec[i] = k_eff.std_dev
print('k_eff_mean: '+str(k_eff.nominal_value))
print('k_eff_dev: '+str(k_eff.std_dev))

k_eff_mean: 0.9456321389465369
k_eff_dev: 0.013753047017155526


In [3]:
x,y,z = Define_Nektar.readNodesFromVtu(file_name)
nodes_dic = {'x':x,'y':y,'z':z}
# Initial temperature distribution in nodes
temp_nodes_vec = Initial_temperature*np.ones(len(x))
fuel_nodes_index = Define_OpenMC.getFuelNodesIndex(nodes_dic,parameters_dic)
# For Test
temp_error_vec = np.zeros(iteration)
temp_ave_error_vec = np.zeros(iteration)

k_eff_mean_vec = np.zeros(iteration)
k_eff_dev_vec = np.zeros(iteration)

heat_ratio = np.zeros(iteration)
flux_ratio = np.zeros(iteration)

num_col = cells_num_dic['n_h']*(cells_num_dic['n_r']+cells_num_dic['n_r_outer'])

heat_mean_mat = np.zeros((iteration,num_col))
heat_dev_mat = np.zeros((iteration,num_col))
flux_mean_mat = np.zeros((iteration,num_col))
flux_dev_mat = np.zeros((iteration,num_col))

In [4]:
# Parameters of heat pipes
fuel_r = parameters_dic['fuel_r']
controlRod_r = parameters_dic['controlRod_r']
heat_pipe_R = parameters_dic['heat_pipe_R']
fuel_h = parameters_dic['fuel_h']

# Get cells mesh 
n_r = cells_num_dic['n_r']
n_r_outer = cells_num_dic['n_r_outer']
n_h = cells_num_dic['n_h']

r_inner_mesh = np.linspace(controlRod_r,(fuel_r-heat_pipe_R),n_r+1)
r_outer_mesh = np.linspace(fuel_r-heat_pipe_R,fuel_r,n_r_outer+1)
r_mesh = np.hstack((r_inner_mesh[0:len(r_inner_mesh)],r_outer_mesh[1:len(r_outer_mesh)]))

h_mesh = np.linspace(-fuel_h/2,fuel_h/2,n_h+1)

# Get the vector of distance
r_axe = np.zeros(n_r+n_r_outer)
r_axe[0:n_r] = (r_mesh[0:n_r]+r_mesh[1:(n_r+1)])/2
r_axe[n_r:(n_r+n_r_outer)]= (r_outer_mesh[0:n_r_outer]+r_outer_mesh[1:(n_r_outer+1)])/2

h_axe = np.zeros(n_h)
h_axe[0:n_h] = (h_mesh[0:n_h]+h_mesh[1:(n_h+1)])/2

In [5]:
np.savetxt('h_axe.txt',h_axe)
np.savetxt('r_axe.txt',r_axe)

In [9]:
fuel_r = parameters_dic['fuel_r']
controlRod_r = parameters_dic['controlRod_r']
heat_pipe_R = parameters_dic['heat_pipe_R']
fuel_h = parameters_dic['fuel_h']

x = nodes_dic['x']
y = nodes_dic['y']
z = nodes_dic['z']

#Get cells mesh 
n_r = cells_num_dic['n_r']
n_r_outer = cells_num_dic['n_r_outer']
n_h = cells_num_dic['n_h']

r_inner_mesh = np.linspace(controlRod_r,(fuel_r-heat_pipe_R),n_r+1)
r_outer_mesh = np.linspace(fuel_r-heat_pipe_R,fuel_r,n_r_outer+1)
r_mesh = np.hstack((r_inner_mesh[0:len(r_inner_mesh)],r_outer_mesh[1:len(r_outer_mesh)]))

h_mesh = np.linspace(-fuel_h/2,fuel_h/2,n_h+1)

# Get the vector of distance
r_axe = np.zeros(n_r+n_r_outer)
r_axe[0:n_r] = (r_mesh[0:n_r]+r_mesh[1:(n_r+1)])/2
r_axe[n_r:(n_r+n_r_outer)]= (r_outer_mesh[0:n_r_outer]+r_outer_mesh[1:(n_r_outer+1)])/2

h_axe = np.zeros(n_h)
h_axe[0:n_h] = (h_mesh[0:n_h]+h_mesh[1:(n_h+1)])/2


In [8]:
for i in range(iteration):
    if os.path.exists('summary.h5'):
        os.remove('summary.h5')
    if os.path.exists('statepoint.20.h5'):
        os.remove('statepoint.20.h5')
    if os.path.exists('tallies.out'):
        os.remove('tallies.out')

    start_tot = time.time()
    
    # Iteration
    print('Iteration: '+str(i+1)+' begins')
    # Run OpenMC !
    time_start = time.time()
    openmc.run(output=False)
    time_end = time.time()
    
    heat_tot_vec = np.zeros(num_col)
    heat_dev_vec = np.zeros(num_col)

    flux_tot_vec = np.zeros(num_col)
    flux_dev_vec = np.zeros(num_col)
    
    sp = openmc.StatePoint('statepoint.{}.h5'.format(settings_dic['batches']))
    k_eff = sp.k_combined
    k_eff_mean_vec[i] = k_eff.nominal_value
    k_eff_dev_vec[i] = k_eff.std_dev
    print('k_eff_mean: '+str(k_eff_mean_vec[i]))
    print('k_eff_dev: '+str(k_eff_dev_vec[i]))
    

    for j in range(len(fuel_cell_ID_list)):
        t = sp.get_tally(name='cell tally '+str(fuel_cell_ID_list[j]))
        heat_tot_vec[j] = t.get_values(scores=['heating'],value='mean').item()
        heat_dev_vec[j] = t.get_values(scores=['heating'],value='std_dev').item()
        flux_tot_vec[j] = t.get_values(scores=['flux'],value='mean').item()
        flux_dev_vec[j] = t.get_values(scores=['flux'],value='std_dev').item()
    del sp

    heat_mean_mat[i,:] = heat_tot_vec
    heat_dev_mat[i,:] = heat_dev_vec
    flux_mean_mat[i,:] = flux_tot_vec
    flux_dev_mat[i,:] = flux_dev_vec
    
    if i>0:
        heat_index = np.where((heat_mean_mat[i,:]<heat_mean_mat[i-1,:]+heat_dev_mat[i-1,:]) & (heat_mean_mat[i,:]>heat_mean_mat[i-1,:]-heat_dev_mat[i-1,:]))
        flux_index = np.where((flux_mean_mat[i,:]<flux_mean_mat[i-1,:]+flux_dev_mat[i-1,:]) & (flux_mean_mat[i,:]>flux_mean_mat[i-1,:]-flux_dev_mat[i-1,:]))
        heat_ratio[i] = len(heat_index[0])/num_col
        flux_ratio[i] = len(flux_index[0])/num_col
    
    print('heat_ratio: '+str(heat_ratio[i]))
    print('flux_ratio: '+str(flux_ratio[i]))

Iteration: 1 begins
k_eff_mean: 0.9914579314521194
k_eff_dev: 0.008731119201406306
heat_ratio: 0.0
flux_ratio: 0.0
Iteration: 2 begins
k_eff_mean: 0.9914579314521189
k_eff_dev: 0.008731119201400211
heat_ratio: 1.0
flux_ratio: 1.0
Iteration: 3 begins
k_eff_mean: 0.9914579314521166
k_eff_dev: 0.008731119201407261
heat_ratio: 1.0
flux_ratio: 1.0
Iteration: 4 begins
k_eff_mean: 0.9914579314521192
k_eff_dev: 0.008731119201403762
heat_ratio: 1.0
flux_ratio: 1.0
Iteration: 5 begins
k_eff_mean: 0.9914579314521145
k_eff_dev: 0.00873111920140198
heat_ratio: 1.0
flux_ratio: 1.0
Iteration: 6 begins
k_eff_mean: 0.9914579314521177
k_eff_dev: 0.008731119201398635
heat_ratio: 1.0
flux_ratio: 1.0
Iteration: 7 begins
k_eff_mean: 0.9914579314521214
k_eff_dev: 0.008731119201402246
heat_ratio: 1.0
flux_ratio: 1.0
Iteration: 8 begins
k_eff_mean: 0.991457931452123
k_eff_dev: 0.008731119201398766
heat_ratio: 1.0
flux_ratio: 1.0
Iteration: 9 begins
k_eff_mean: 0.9914579314521192
k_eff_dev: 0.008731119201403762

In [4]:
for i in range(iteration):
    start_tot = time.time()
    # For Test
    temp_cells_mat_last = temp_cells_mat
    
    # Iteration
    print('Iteration: '+str(i+1)+' begins')
    # Run OpenMC !
    time_start = time.time()
    openmc.run(output=False)
    time_end = time.time()
    print('Iteration: '+str(i+1)+' OpenMC run time:'+str(time_end-time_start))
    
    # Post-process the result of heat source and generate Force.pts
    time_start = time.time()
    k_eff,tally_dic = Define_OpenMC.postProcess(nodes_dic,volume_mat,temp_nodes_vec,fuel_nodes_index,parameters_dic,cells_num_dic,settings_dic,fuel_cell_ID_list)
    time_end = time.time()
    print('Iteration: '+str(i+1)+' OpenMC post-process time:'+str(time_end-time_start))
    
    # k-eff: mean value and standard deviation
    k_eff_mean_vec[i] = k_eff.nominal_value
    k_eff_dev_vec[i] = k_eff.std_dev
    print('k_eff_mean: '+str(k_eff_mean_vec[i]))
    print('k_eff_dev: '+str(k_eff_dev_vec[i]))
    
#     # Heat and flux
#     heat_mean_vec = tally_dic['heat_mean']
#     heat_dev_vec = tally_dic['heat_dev']
#     flux_mean_vec = tally_dic['flux_mean']
#     flux_dev_vec = tally_dic['flux_dev']
    
    heat_mean_mat[i,:] = tally_dic['heat_mean']
    heat_dev_mat[i,:] = tally_dic['heat_dev']
    flux_mean_mat[i,:] = tally_dic['flux_mean']
    flux_dev_mat[i,:] = tally_dic['flux_dev']
    
    if i>0:
        heat_index = np.where((heat_mean_mat[i,:]<heat_mean_mat[i-1,:]+heat_dev_mat[i-1,:]) & (heat_mean_mat[i,:]>heat_mean_mat[i-1,:]-heat_dev_mat[i-1,:]))
        flux_index = np.where((flux_mean_mat[i,:]<flux_mean_mat[i-1,:]+flux_dev_mat[i-1,:]) & (flux_mean_mat[i,:]>flux_mean_mat[i-1,:]-flux_dev_mat[i-1,:]))
        heat_ratio[i] = len(heat_index[0])/num_col
        flux_ratio[i] = len(flux_index[0])/num_col
    
    print('heat_ratio: '+str(heat_ratio[i]))
    print('flux_ratio: '+str(flux_ratio[i]))
    
    # Run Nektar++
    if os.path.exists(file_name+'.fld'):
        os.remove(file_name+'.fld')
    if os.path.exists(file_name+'.vtu'):
        os.remove(file_name+'.vtu')    
    
    time_start = time.time()
    file_name_new = Define_Nektar.runNektar_Temp(file_name,solver_name,temp_pipe,i)
    time_end = time.time()
    print('Iteration: '+str(i+1)+' Nektar run time:'+str(time_end-time_start))
        
    # Post-process result of temperature
    time_start = time.time()
    temp_nodes_vec = Define_Nektar.postProcess_Temp(file_name_new)
    time_end = time.time()
    print('Iteration: '+str(i+1)+' Nektar post-process time:'+str(time_end-time_start))
    temp_cells_mat = Define_OpenMC.getCellTemperature(nodes_dic,temp_nodes_vec,fuel_nodes_index,parameters_dic,cells_num_dic)

    volume_mat,fuel_cell_ID_list = Define_OpenMC.define_Geo_Mat_Set(cells_num_dic,parameters_dic,settings_dic,temp_cells_mat,controlRod_deep)
    # For Test
    
    # Largest relative error of fuel temperature
    temp_error = np.abs(temp_cells_mat-temp_cells_mat_last)/temp_cells_mat_last
    temp_error_vec[i] = temp_error.max()
    print('Maximum relative error: '+str(temp_error_vec[i]))
    
    temp_ave_error_vec[i] = temp_error.sum()/(np.size(temp_error,0)*np.size(temp_error,1))
    print('Average relative error: '+str(temp_ave_error_vec[i]))
    
    end_tot = time.time()
    print('Total time: '+str(end_tot-start_tot))
    
#     if (heat_ratio[i]>=0.95) | (flux_ratio[i]>=0.95):
#         break

Iteration: 1 begins
Iteration: 1 OpenMC run time:20.40781259536743
Iteration: 1 OpenMC post-process time:34.08793354034424
k_eff_mean: 0.98539699772723
k_eff_dev: 0.018362041923381173
heat_ratio: 0.0
flux_ratio: 0.0
Iteration: 1 Nektar run time:204.8906831741333
Iteration: 1 Nektar post-process time:2.0515594482421875
Maximum relative error: 0.03279367365315959
Average relative error: 0.017838676477300593
Total time: 265.6520049571991
Iteration: 2 begins
Iteration: 2 OpenMC run time:18.445114612579346
Iteration: 2 OpenMC post-process time:21.518826246261597
k_eff_mean: 0.9988541241618776
k_eff_dev: 0.009413046853437851
heat_ratio: 0.4533333333333333
flux_ratio: 0.44333333333333336
Iteration: 2 Nektar run time:204.89015102386475
Iteration: 2 Nektar post-process time:1.9813790321350098
Maximum relative error: 0.0012664141193764625
Average relative error: 0.0004937469490344518
Total time: 250.936274766922
Iteration: 3 begins
Iteration: 3 OpenMC run time:16.74135994911194
Iteration: 3 Open

In [5]:
temp_error_vec

array([0.03279367, 0.00126641, 0.00093275, 0.00147653, 0.00095112,
       0.00138603, 0.00227054, 0.00078839, 0.00113817, 0.00090424])

In [5]:
k_eff_mean_vec

array([0.985397  , 0.99885412, 1.00381561, 1.00189723, 1.00163026,
       1.00517517, 1.00001472, 1.00254461, 1.00677473, 1.009172  ])

In [7]:
np.savetxt('heat_mean_mat.txt',heat_mean_mat)

In [6]:
type(volume_mat)

numpy.ndarray

In [7]:
np.savetxt('volume_mat.txt',volume_mat)